# 1. Importação das bibliotecas 

Aqui é feito a importação de todos os pacotes que serão utilizado nos estudos

In [ ]:
import os
import string
import unicodedata
import json
import multiprocessing
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold , RandomizedSearchCV , GridSearchCV , cross_val_score, KFold
from sklearn.metrics import classification_report, accuracy_score , make_scorer, r2_score , f1_score , confusion_matrix
from scipy.stats import randint, uniform
from joblib import dump, load
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.models import Sequential , clone_model , load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.ensemble import RandomForestClassifier

# 2.  Configuração das bibliotecas

essa linha abaixo serve para resetar a largura das colunas na hora de exibir, em alguns cenarios estava tendo bug e tive que usar isso para adaptar

In [ ]:
pd.reset_option('display.max_colwidth')

# 3. importação dos CSV e transformando em datasets pandas

In [ ]:
gruposDFnomeDasColunas = [
    "nome",
    "classificacao",
    "frequencia_feminina",
    "frequencia_masculina",
    "frequencia_total",
    "proporcao",
    "nomes_alternativos"    
]
gruposDF = pd.read_csv('../data/grupos.csv', names=gruposDFnomeDasColunas, header=0)
gruposDF.head()

In [ ]:
nomesDSnomeDasColunas = [
    "nomes_alternativos",
    "classificacao",
    "primeiro_nome",
    "frequencia_feminina",
    "frequencia_masculina",
    "frequencia_total",
    "frequencia_grupo",
    "nome_grupo",
    "proporcao"
]
nomesDF= pd.read_csv("../data/nomes.csv", names=nomesDSnomeDasColunas, header=0)
nomesDF.head()

# 4. Limpeza de nulos e n/a

nas limpezas dos dados nulos, foram verificados que os dados "NAN" na verdade, são dados que vazios, pois por exemplo, na linha AALINE, temos a frequencia_feminina de 66 e  frequencia_total de 66 também, então sobraria 0 para a frequencia_masculina

In [ ]:
gruposDF.fillna(0, inplace=True)
nomesDF.fillna(0, inplace=True)

gruposDF.drop_duplicates(inplace=True)
nomesDF.drop_duplicates(inplace=True)

# 5. Criando novos dados 

## 5.1 Porcentagem de cada classe
para melhor visualização da frequencia, é necessario a criação dos dados de porcentagem 

In [ ]:
gruposDF.head()

In [ ]:
# gruposDF.drop(columns=["nomes_alternativos"],inplace=True)
# nomesDF.drop(columns=["nomes_alternativos","frequencia_grupo","nome_grupo"],inplace=True)

In [ ]:
gruposDF["porcentagem_feminina"]  = 0
gruposDF["porcentagem_masculina"] = 0
nomesDF["porcentagem_feminina"] = 0
nomesDF["porcentagem_masculina"] = 0

In [ ]:
nomesDF.rename(columns=
               {"primeiro_nome": "nome"}
               ,inplace=True)

In [ ]:
gruposDF["porcentagem_masculina"] = round(gruposDF["frequencia_masculina"] / gruposDF["frequencia_total"], 7)
gruposDF["porcentagem_feminina"] =  round(gruposDF["frequencia_feminina"]  / gruposDF["frequencia_total"], 7)
    

In [ ]:
gruposDF.head()

In [ ]:
nomesDF["porcentagem_masculina"] = round(nomesDF["frequencia_masculina"] / nomesDF["frequencia_total"], 7)
nomesDF["porcentagem_feminina"] = round(nomesDF["frequencia_feminina"] / nomesDF["frequencia_total"], 7)
    

In [ ]:
nomesDF[(nomesDF["porcentagem_feminina"] == 0) & (nomesDF["porcentagem_masculina"] == 0)]

In [ ]:
gruposDF[(gruposDF["porcentagem_feminina"] == 0) & (gruposDF["porcentagem_masculina"] == 0)]

In [ ]:
nomesDF.set_index('nome', inplace=True)
gruposDF.set_index('nome', inplace=True)

In [ ]:
data = gruposDF.combine_first(nomesDF).reset_index()

In [ ]:
data.head()

## 5.2 Criando colunas binarias

Para alimentar os modelos, foi pensado a estrategia de criar colunas binarias para cada posição do nome , por exemplo, 

a letra 1 ( primeira letra) é igual a A ? ou seja , a coluna LETRA_1_A , e assim por diante 


In [ ]:
def processar_dataset(data, max_posicoes=20):
    alfabeto = string.ascii_uppercase
    novas_colunas = [f"LETRA_{i}_{letra}" for i in range(1, max_posicoes + 1) for letra in alfabeto]
    novas_colunas_df = pd.DataFrame(0, index=data.index, columns=novas_colunas)
    data = pd.concat([data, novas_colunas_df], axis=1)
    for index, row in data.iterrows():
        nome = row['nome'].upper()
        for posicao, letra in enumerate(nome):
            if posicao < max_posicoes and letra in alfabeto:
                coluna = f"LETRA_{posicao + 1}_{letra}"
                if coluna in data.columns:
                    data.at[index, coluna] = 1

    return data
data = processar_dataset(data)
data

## 6. Criando e executando label encoder
o label Encoder faz a função de codificador , para transformar os dados qualitativos em numeros, para assim que a maquina prever , o label encoder poder utilizar a função de inverse transform e trazer a real classe da previsão 

In [ ]:
le = LabelEncoder()
le = le.fit(data['classificacao'])
data['classificacao'] = le.fit_transform(data['classificacao'])
data.head()

# separando dados de treino e teste
separa as colunas de input e a coluna de target (Classificacao) 

In [ ]:
X = data.drop(columns=["nome",	'frequencia_grupo','nome_grupo','nomes_alternativos',"classificacao"	,"frequencia_feminina",	"frequencia_masculina"	,"frequencia_total"	,"proporcao"	,"porcentagem_feminina"	,"porcentagem_masculina"])
y = data['classificacao']

# 7. Criando Funções Auxiliares

## 7.1 Criando função de input de dados
a função de input de dados recebe um nome, e tranforma em colunas binarias, assim como os dados de treino, para encaixar com o shape dos dados

In [ ]:
def preparar_input_para_modelo(nome, max_posicoes=20):
    nome = nome.upper()
    alfabeto = string.ascii_uppercase
    input_vector = np.zeros(max_posicoes * len(alfabeto), dtype=int)
    for posicao, letra in enumerate(nome):
        if posicao < max_posicoes and letra in alfabeto:
            indice_letra = alfabeto.index(letra)
            indice_vetor = posicao * len(alfabeto) + indice_letra
            input_vector[indice_vetor] = 1

    
    return input_vector.reshape(1, -1)

## 7.2 Criando Função para plotar Matriz de Confusão 

essa vai ser uma função auxiliar que vai nos ajudar a evitar repetição de codigo

In [ ]:
def plotar_matriz_confusao(cm, classes=['F', 'M'], title='Matriz de Confusão', cmap='Blues'):
    plt.figure(figsize=(10, 7))
    ax = sns.heatmap(cm, annot=False, fmt='d', cmap=cmap, xticklabels=classes, yticklabels=classes, cbar=False)
    
    # Adicionar anotações de TP, FP, FN, TN
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if i == j:
                annotation = f'{cm[i, j]} (TP)'  # Verdadeiro Positivo
            elif i > j:
                annotation = f'{cm[i, j]} (FN)'  # Falso Negativo
            else:
                annotation = f'{cm[i, j]} (FP)'  # Falso Positivo
            ax.text(j + 0.5, i + 0.5, annotation, color='black', ha='center', va='center')

    plt.xlabel('Previsto')
    plt.ylabel('Valor Real')
    plt.title(title)
    plt.show()


## 7.3 Criando Função Auxiliar  para criar diretorios dinamicos 

isso ajudara a organizar as pastas

In [ ]:
def criar_diretorio(caminho):
    if not os.path.exists(caminho):
        os.makedirs(caminho)

## 7.4 Criando Objeto Auxiliar para manter todos os modelos 

esse objeto servirar para manter todos os modelos salvos em um lugar para testar de forma manual futuramente 

In [ ]:
modelos = {}

## 7.5 Criação Função Auxiliar para gerar relatorios de classificação
essa função auxiliar ajuda a evitar repetição de codigo toda vez que for gerar um relatorio de classificação

In [ ]:
def gerar_relatorio_classificacao(Y_teste, previsoes, le=le):
    Y_teste_decodificado = le.inverse_transform(Y_teste)
    previsoes_decodificadas = le.inverse_transform(previsoes)
    class_report = classification_report(Y_teste_decodificado, previsoes_decodificadas,output_dict=True)
    print(class_report)
    print(f'Acuracia {class_report["accuracy"]}')
    return class_report, Y_teste_decodificado, previsoes_decodificadas


## 7.6 Criação de função para treinar|Carregar modelos "simples"
essa é a principal das funções auxiliares, ela é responsavel por treinar ou carregar os dados dos modelos do scikit learn e consumir as outras funções auxiliares para trazer os dados 

In [ ]:
def treinar_e_avaliar_modelo(modelo, X, y, model_name, classes=['F', 'M'],modelos=modelos):
    base_path = f'../modelos_e_resultados/{model_name}/'
    model_path = os.path.join(base_path, 'modelo.joblib')
    results_path = os.path.join(base_path, 'resultados.json')
    
    # Criar diretório para o modelo se não existir
    criar_diretorio(base_path)
    
    if os.path.exists(model_path) and os.path.exists(results_path):
        # Carregar modelo e resultados
        modelo = load(model_path)
        with open(results_path, 'r') as f:
            resultados = json.load(f)
        
        # Exibir os resultados carregados
        print(f"Resultados carregados:")
        print(f"F1-scores de cada fold: {resultados['f1_scores']}")
        print(f"Média do F1-score: {resultados['media_f1_score']}")
        print("Classification Report:")
        print(resultados['classification_report'])
        
        # Exibir a matriz de confusão carregada
        cm = np.array(resultados['confusion_matrix'])
        plotar_matriz_confusao(cm, classes=classes)
    else:
        # Treinar o modelo e calcular os resultados
        X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        
        kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        f1_scorer = make_scorer(f1_score, average='macro')
        f1_scores = cross_val_score(modelo, X_treino, Y_treino, cv=kf, scoring=f1_scorer)
        
        print(f'F1-scores de cada fold: {f1_scores}')
        print(f'Média do F1-score: {f1_scores.mean()}')
        
        
        modelo.fit(X_treino, Y_treino)
        dump(modelo, model_path)
        
        previsoes = modelo.predict(X_teste)
        acc = accuracy_score(Y_teste, previsoes)
        print(f'{model_name} - Acurácia: {acc}')
        
   
        class_report, Y_teste_decodificado, previsoes_decodificadas = gerar_relatorio_classificacao(Y_teste, previsoes, le)
        
    
        cm = confusion_matrix(Y_teste_decodificado, previsoes_decodificadas)
        plotar_matriz_confusao(cm, classes=classes)
        modelos[model_name] = modelo
   
        resultados = {
            "f1_scores": f1_scores.tolist(),
            "media_f1_score": f1_scores.mean(),
            "classification_report": class_report,
            "confusion_matrix": cm.tolist()
        }
        with open(results_path, 'w') as f:
            json.dump(resultados, f)
    
    return modelo

## 7.8 Criação da função auxiliar para remover acentos de nomes

In [ ]:
def remove_acentos(string):
    nfkd = unicodedata.normalize('NFKD', string)
    return "".join(c for c in nfkd if not unicodedata.combining(c))


## 7.9 Criação da função Auxiliar para buscar nome no dataframe

In [ ]:
def busca_no_dataframe(nome):
    nome = nome.upper()
    if nome in data['nome'].values:        
        copyDT = data.loc[data['nome'] == nome, [
            'nome', 'classificacao', 'frequencia_feminina', 'nome_grupo',
            'frequencia_masculina', 'frequencia_total',
            'proporcao', 'porcentagem_feminina',
            'porcentagem_masculina', 'nomes_alternativos'  # Incluindo a nova coluna
        ]].copy()

        
        retornoDataFrame = copyDT.to_dict(orient='records')[0] 
        

        if 'nomes_alternativos' in retornoDataFrame and pd.notna(retornoDataFrame['nomes_alternativos']):
            retornoDataFrame['nomes_alternativos'] = retornoDataFrame['nomes_alternativos'].split('|')
        else:
            retornoDataFrame['nomes_alternativos'] = []  
        

        retornoDataFrame['status'] = '200'
        retornoDataFrame['classificacao'] = le.inverse_transform([retornoDataFrame['classificacao']])[0]

        return retornoDataFrame 
    else:
        return {"status": "400"}


resultado = busca_no_dataframe('augusto')
print(resultado)


In [ ]:
data['nomes_alternativos']

## 7.10  Criando função auxiliar para desconverter de classeEncode para classeOriginal

In [ ]:
def desconverteEncoding(resultado, le=le):
    return le.inverse_transform([resultado])[0]
    

- - - 

# 8. Regressão Logistica

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg = treinar_e_avaliar_modelo(log_reg, X, y, "regressao_logistica")
modelos["regressao_logistica"] = log_reg
log_reg





Com o modelo de regressao logistica foi obtido o resultado medio de F1-score de ~0.85, que significa que temos um um desempenho constante mesmo com diferentes conjuntos de dados . 

Já o classification_report nos da a informação que a precisão para ambas as classes é de 0.87, o que é um otimo sinal que não há um overfit pois é um valor bem proximo do F1 score

a matriz de confusão também revela que há valores muito baixo de Falso Negativos e Falso Positivos comparados ao Verdadeiro Positivo e ao Verdadeiro Negativo

# 9. KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn = treinar_e_avaliar_modelo(knn, X, y, "knn")
modelos['knn'] = knn
knn

No resultado do knn vemos que o f1 score dele foi um pouco maior que o anterior (regressao logistica ) porem os falsos positivos e falsos negativos foram bem maiores 

# 10. Naive bayes | GaussianNB

In [ ]:
naive_bayes = GaussianNB()
naive_bayes = treinar_e_avaliar_modelo(naive_bayes, X, y, "naive_bayes")
modelos['naive_bayes'] = naive_bayes
naive_bayes

o Modelo de naive bayes foi bem inferior em questão do F1-Score em relação aos dois ultimos modelos  e tivemos um crescente muito grande em questão de falsos negativos

# 11. Random forest

In [ ]:
random_forest = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=42)
random_forest = treinar_e_avaliar_modelo(random_forest, X, y, "random_forest")
modelos['random_forest'] = random_forest
random_forest

o modelo de random forest foi o melhor de de F1-score até agora, e obteve um falso negativo baixo , e obtemos valores medianos bem baixo tambem de falso postivo

# 12. rede neural

## 12.1 Criando função especifica para o tensorflow/keras

In [ ]:
def criar_modelo_neural(input_dim):
    modelo = Sequential([
        Dense(160, input_dim=input_dim, activation='relu'), 
        Dropout(0.2),
        Dense(80, activation='relu'), 
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return modelo

In [ ]:
def treinar_e_avaliar_modelo_keras(modelo, X, y, model_name, classes=['F', 'M'], epochs=50, batch_size=32, n_splits=5):

    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values

    X = np.array(X)
    y = np.array(y)


    base_path = f'../modelos_e_resultados/{model_name}/'
    model_path = os.path.join(base_path, 'modelo.h5')
    results_path = os.path.join(base_path, 'resultados.json')

  
    os.makedirs(base_path, exist_ok=True)

    if os.path.exists(model_path) and os.path.exists(results_path):
      
        modelo = load_model(model_path)
        with open(results_path, 'r') as f:
            resultados = json.load(f)

        print("Resultados carregados:")
        print(f"Acurácias de cada fold: {resultados['accuracies']}")
        print(f"Média da Acurácia: {resultados['media_accuracia']}")
        print("Classification Report:")
        print(resultados['classification_report'])

      
        cm = np.array(resultados['confusion_matrix'])
        plotar_matriz_confusao(cm, classes=classes)
    else:
    
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

  
        historicos = []
        accuracies = []
        melhor_acuracia = 0
        melhor_modelo = None

     
        for fold, (train_index, val_index) in enumerate(kf.split(X, y), 1):
            print(f"\nTreinando fold {fold}")

            
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

           
            model_clone = clone_model(modelo)
            model_clone.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


            history = model_clone.fit(X_train, y_train,
                                      validation_data=(X_val, y_val),
                                      epochs=epochs,
                                      batch_size=batch_size,
                                      verbose=1)

       
            _, accuracy = model_clone.evaluate(X_val, y_val, verbose=0)
            accuracies.append(accuracy)
            historicos.append(history.history)

            print(f"Acurácia do fold {fold}: {accuracy}")

      
            if accuracy > melhor_acuracia:
                melhor_acuracia = accuracy
                melhor_modelo = model_clone


        melhor_modelo.save(model_path)


        y_pred = melhor_modelo.predict(X)
        y_pred_classes = (y_pred > 0.5).astype(int).flatten()


        class_report = classification_report(y, y_pred_classes, target_names=classes)
        cm = confusion_matrix(y, y_pred_classes)


        plotar_matriz_confusao(cm, classes=classes)


        resultados = {
            "accuracies": accuracies,
            "media_accuracia": np.mean(accuracies),
            "classification_report": class_report,
            "confusion_matrix": cm.tolist(),
            "historicos": historicos
        }

        with open(results_path, 'w') as f:
            json.dump(resultados, f)

        print("\nResultados finais:")
        print(f"Acurácias de cada fold: {accuracies}")
        print(f"Média da Acurácia: {np.mean(accuracies)}")
        print("\nClassification Report:")
        print(class_report)

        modelos[model_name] = modelo

    return modelo



## 12.2 Modelo de rede neural

In [ ]:
rede = criar_modelo_neural(X.shape[1])
rede.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rede = treinar_e_avaliar_modelo_keras(rede,X,y,'rede_neural')
modelos['rede_neural'] = rede

nesse modelo utilizando rede neurais , é possivel ver que a acuracia media de cada fold é bme alta (95%) e que no melhor modelo chega a bater um f1-score de 0.99% , o numero de falsos positivos é maior que alguns modelos e tem o  menor nuemro de falso negativo até agora

# 13. Algoritmos de conjunto

## 13.1  Bagging com rede neural

In [ ]:
def bagging_neural_network_kfold(X, y, model_name, classes=['F', 'M'], n_estimators=10, epochs=50, batch_size=32, n_splits=5):
    base_path = f'../modelos_e_resultados/{model_name}/'
    results_path = os.path.join(base_path, 'resultados.json')
    
    criar_diretorio(base_path)
    
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, pd.Series):
        y = y.values

    X = np.array(X)
    y = np.array(y)
    
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    accuracies = []
    f1_scores = []
    all_y_true = []
    all_y_pred = []
    modelos_bagging = []

    for fold, (train_index, val_index) in enumerate(kf.split(X, y_encoded), 1):
        print(f"\nProcessando fold {fold}/{n_splits}")
        
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y_encoded[train_index], y_encoded[val_index]
        
        best_acc = 0
        best_f1 = 0
        best_model_path = None
        fold_models = []
        
        for i in range(n_estimators):
            model_path = os.path.join(base_path, f'modelo_fold{fold}_estimador{i+1}.h5')
            
            if os.path.exists(model_path):
                print(f"Carregando modelo {i+1}/{n_estimators} do fold {fold} de: {model_path}")
                modelo = load_model(model_path)
            else:
                print(f"Treinando modelo {i+1}/{n_estimators} no fold {fold}")
                indices = np.random.choice(X_train.shape[0], X_train.shape[0], replace=True)
                X_bootstrap = X_train[indices]
                y_bootstrap = y_train[indices]
                
                modelo = criar_modelo_neural(X.shape[1])
                modelo.fit(X_bootstrap, y_bootstrap, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                
                modelo.save(model_path)
                print(f"Modelo salvo em: {model_path}")
            
            previsoes = modelo.predict(X_val).flatten()
            previsoes_finais = (previsoes > 0.5).astype(int)
            
            acc = accuracy_score(y_val, previsoes_finais)
            f1 = f1_score(y_val, previsoes_finais, average='macro')

            if f1 > best_f1:
                best_acc = acc
                best_f1 = f1
                best_model_path = model_path
            
            fold_models.append(modelo)
        
        print(f"Melhor modelo do fold {fold}: {best_model_path} com F1-score de {best_f1:.2f}")
        
        best_model = load_model(best_model_path)
        previsoes = best_model.predict(X_val).flatten()
        previsoes_finais = (previsoes > 0.5).astype(int)
        
        accuracies.append(best_acc)
        f1_scores.append(best_f1)
        
        all_y_true.extend(y_val)
        all_y_pred.extend(previsoes_finais)
        
        print(f"Acurácia do fold {fold}: {best_acc:.2f}")
        print(f"F1-score do fold {fold}: {best_f1:.2f}")
        
        modelos_bagging.extend(fold_models)
    
    class_report, all_y_true_decoded, all_y_pred_decoded = gerar_relatorio_classificacao(np.array(all_y_true), np.array(all_y_pred), le)
    
    cm = confusion_matrix(all_y_true_decoded, all_y_pred_decoded)
    plotar_matriz_confusao(cm, classes=classes)
    
    resultados = {
        "accuracies": accuracies,
        "media_accuracia": np.mean(accuracies),
        "f1_scores": f1_scores,
        "media_f1_score": np.mean(f1_scores),
        "classification_report": class_report,
        "confusion_matrix": cm.tolist()
    }
    
    with open(results_path, 'w') as f:
        json.dump(resultados, f)
    
    print("\nResultados finais:")
    print(f"Acurácias de cada fold: {accuracies}")
    print(f"Média da Acurácia: {np.mean(accuracies)}")
    print(f"F1-scores de cada fold: {f1_scores}")
    print(f"Média do F1-score: {np.mean(f1_scores)}")
    print("\nClassification Report:")
    print(class_report)
    
    return modelos_bagging

In [ ]:
modelos_bagging = bagging_neural_network_kfold(X, y, model_name='bagging_kfold', classes=['F', 'M'])

In [ ]:
def previsao_bagging(modelos, X):
    if len(modelos) == 0:
        raise ValueError("A lista de modelos está vazia. Certifique-se de que os modelos estão treinados e adicionados à lista.")
    
    previsoes_agregadas = np.zeros(X.shape[0])
    
    for modelo in modelos:
        previsoes = modelo.predict(X).flatten()
        previsoes_agregadas += previsoes

    previsoes_agregadas /= len(modelos)
    
    previsoes_finais = (previsoes_agregadas > 0.5).astype(int)
    
    return previsoes_finais

o modelo de baggins de rede neurais demonstra ter mais falsos positivos e negativos do que a propria rede neural isolada, porém apresenta um f1 score bom

## 13.2 Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np


In [ ]:
def treinar_e_avaliar_adaboost_logreg(X, y, model_name, classes=['F', 'M']):
    base_path = f'../modelos_e_resultados/{model_name}/'
    model_path = os.path.join(base_path, 'modelo_adaboost_logreg.joblib')
    results_path = os.path.join(base_path, 'resultados.json')

    criar_diretorio(base_path)

    if os.path.exists(model_path) and os.path.exists(results_path):
        adaboost_custom = load(model_path)
        with open(results_path, 'r') as f:
            resultados = json.load(f)


        print(f"Resultados carregados:")
        print(f"Acurácia: {resultados['accuracy']:.2f}")
        print(resultados['classification_report'])


        cm = np.array(resultados['confusion_matrix'])
        plotar_matriz_confusao(cm, classes=classes)
    else:

        X = np.array(X)
        y = np.array(y)


        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


        estimator = LogisticRegression(max_iter=1000)


        adaboost_custom = AdaBoostClassifier(estimator=estimator, n_estimators=50, random_state=42)


        adaboost_custom.fit(X_train, y_train)


        y_pred = adaboost_custom.predict(X_test)


        accuracy = accuracy_score(y_test, y_pred)
        print(f"Acurácia: {accuracy:.2f}")

        class_report = gerar_relatorio_classificacao(y_test, y_pred)


        cm = confusion_matrix(y_test, y_pred)
        plotar_matriz_confusao(cm, classes=classes)


        dump(adaboost_custom, model_path)
        print(f'class report type {type(class_report[1])}')
    
        resultados = {
            "accuracy": accuracy,
            "classification_report": class_report[0],
            "confusion_matrix": cm.tolist()
        }
        with open(results_path, 'w') as f:
            json.dump(resultados, f)

    return adaboost_custom


In [ ]:
adaboost_model = treinar_e_avaliar_adaboost_logreg(X, y, 'boosting_log_reg')
adaboost_model

o modelo de adaboost com regressao logistica apresentou uma acuracia relativamente baixa comparado aos outros modelos , valores altos de falso positivo e negativos , e um f1 score baixo também

## 13.4 Stacking

In [ ]:
def treinar_e_avaliar_stacking(X, y, model_name, classes=['Class 0', 'Class 1']):
    base_path = f'../modelos_e_resultados/{model_name}/'
    model_path = os.path.join(base_path, 'modelo_stacking.joblib')
    results_path = os.path.join(base_path, 'resultados.json')

    criar_diretorio(base_path)

    if os.path.exists(model_path) and os.path.exists(results_path):

        stacking_model = load(model_path)
        with open(results_path, 'r') as f:
            resultados = json.load(f)


        print(f"Resultados carregados:")
        print(f"Acurácia: {resultados['accuracy']:.2f}")
        print(resultados['classification_report'])


        cm = np.array(resultados['confusion_matrix'])
        plotar_matriz_confusao(cm, classes=classes)
    else:

        X = np.array(X)
        y = np.array(y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

 
        estimators = [
            ('dt', DecisionTreeClassifier(max_depth=3)),
            ('rf', RandomForestClassifier(n_estimators=100))
        ]


        meta_model = LogisticRegression(max_iter=1000)


        stacking_model = StackingClassifier(estimators=estimators, final_estimator=meta_model)


        stacking_model.fit(X_train, y_train)


        y_pred = stacking_model.predict(X_test)


        accuracy = accuracy_score(y_test, y_pred)
        print(f"Acurácia: {accuracy:.2f}")

        class_report = gerar_relatorio_classificacao(y_test, y_pred)


        cm = confusion_matrix(y_test, y_pred)
        plotar_matriz_confusao(cm, classes=classes)

        dump(stacking_model, model_path)

        resultados = {
            "accuracy": accuracy,
            "classification_report": class_report[0],
            "confusion_matrix": cm.tolist()
        }
        print(resultados)
        with open(results_path, 'w') as f:
            json.dump(resultados, f)

    return stacking_model

In [ ]:
stacking_model = treinar_e_avaliar_stacking(X, y, 'stacking_model')
stacking_model 

o algoritmo de staciking de apresentou resultado semelhante ao de boosting, também com valores de acuracia e f1 baixos e classificações erradas 

In [ ]:
def preveEmTodosModelos(nome):
    retorno = {}
    
    nome = remove_acentos(nome)
    inputDados = preparar_input_para_modelo(nome)
    print("Convertendo input de dados")
    print(inputDados)
    print(inputDados.shape)
    print("-------------------")
    
    if not ('log_reg' in globals()):
        raise ValueError("o modelo de regressão logistica não foi carregado.")
    else:
        resultadoLogReg = desconverteEncoding(log_reg.predict(inputDados))
        retorno['LogReg'] = resultadoLogReg
        print(f'Resultado LogReg : {resultadoLogReg}')
        print("--------------------------")

    if not ('knn' in globals()):
        raise ValueError("o modelo de KNN não foi carregado.")
    else:
        resultadoKnn = desconverteEncoding(knn.predict(inputDados))
        retorno['knn'] = resultadoKnn
        print(f'Resultado KNN : {resultadoKnn}')
        print("--------------------------")

    if not ('naive_bayes' in globals()):
        raise ValueError("o modelo de naive_bayes não foi carregado.")
    else:
        resultadoNaive_bayes = desconverteEncoding(naive_bayes.predict(inputDados))
        retorno['naive_bayes'] = resultadoNaive_bayes
        print(f'Resultado naive_bayes : {resultadoNaive_bayes}')
        print("--------------------------")

    if not ('random_forest' in globals()):
        raise ValueError("o modelo de random_forest não foi carregado.")
    else:
        resultadoRandom_forest = desconverteEncoding(random_forest.predict(inputDados))
        retorno['random_forest'] = resultadoRandom_forest
        print(f'Resultado random_forest : {resultadoRandom_forest}')
        print("--------------------------")

    if not ('rede' in globals()):
        raise ValueError("o modelo de redeNeural não foi carregado.")
    else:
        resultadoRede = rede.predict(inputDados)
        resultadoRede = desconverteEncoding(int(resultadoRede[0]))
        retorno['rede_neural'] = resultadoRede
        print(f'Resultado redeNeural {resultadoRede}')
        print("--------------------------")

    if not ('modelos_bagging' in globals()):
        raise ValueError("o modelo de baggin não foi carregado.")
    else:
        print(f'modelos_bagging : {modelos_bagging}')
        resultadoBagging = previsao_bagging(modelos_bagging, inputDados)
        resultadoBagging = desconverteEncoding(resultadoBagging)
        retorno['modelos_bagging'] = resultadoBagging
        print(f'Resultado bagging {resultadoBagging}')
        print("--------------------------")

    if not ('adaboost_model' in globals()):
        raise ValueError("o modelo de adaboost_model não foi carregado.")
    else:
        resultadoBoosting = desconverteEncoding(adaboost_model.predict(inputDados))
        retorno['resultadoBoosting'] = resultadoBoosting
        print(f'Resultado BoostingAda {resultadoBoosting}')
        print("--------------------------")

    if not ('stacking_model' in globals()):
        raise ValueError("o modelo de stacking_model não foi carregado.")
    else:
        resultadoStacking = desconverteEncoding(stacking_model.predict(inputDados))
        retorno['stacking_model'] = resultadoStacking
        print(f'Resultado Stacking {resultadoStacking}')
        print("--------------------------")

    if not ('data' in globals()):
        raise ValueError("o dataframe nao foi carregado.")
    else:
        resultadoDataFrame = busca_no_dataframe(nome)
        retorno['data'] = resultadoDataFrame
        print(f'Resultado Stacking {resultadoDataFrame}')
        print("--------------------------")

    return retorno

In [ ]:
preveEmTodosModelos('ariel')

In [ ]:
diretorio='../modelos_e_resultados/'
resultados_modelos = {}


for nome_pasta in os.listdir(diretorio):
    caminho_pasta = os.path.join(diretorio, nome_pasta)

    # Verificar se é uma pasta
    if os.path.isdir(caminho_pasta):
        caminho_json = os.path.join(caminho_pasta, 'resultados.json')

            # Verificar se o arquivo JSON existe
        if os.path.exists(caminho_json):
            with open(caminho_json, 'r') as arquivo:
                resultados_modelos[nome_pasta] = json.load(arquivo)
resultados = resultados_modelos
for modelo, dados in resultados.items():
    print(f"Modelo: {modelo}")
    print("Dados carregados:")
    print(dados)
    print("-" * 40)

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS  
import os
import json

app = Flask(__name__)
CORS(app)  

diretorio = '../modelos_e_resultados/'
resultados_modelos = {}

for nome_pasta in os.listdir(diretorio):
    caminho_pasta = os.path.join(diretorio, nome_pasta)
    if os.path.isdir(caminho_pasta):
        caminho_json = os.path.join(caminho_pasta, 'resultados.json')
        if os.path.exists(caminho_json):
            with open(caminho_json, 'r') as arquivo:
                resultados_modelos[nome_pasta] = json.load(arquivo)

@app.route('/prever', methods=['GET'])
def prever():
    nome = request.args.get('nome')
    if not nome:
        return jsonify({"error": "Nome não fornecido"}), 400
    resultado = preveEmTodosModelos(nome)
    return jsonify(resultado)

@app.route('/buscar', methods=['GET'])
def buscar():
    nome = request.args.get('nome')
    if not nome:
        return jsonify({"error": "Nome não fornecido"}), 400
    resultado = busca_no_dataframe(nome)
    return jsonify(resultado)

@app.route('/resultados', methods=['GET'])
def resultados():
    return jsonify(resultados_modelos)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)
